<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/evaluation/BeirEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BEIR Out of Domain Benchmark

About [BEIR](https://github.com/beir-cellar/beir):

BEIR is a heterogeneous benchmark containing diverse IR tasks. It also provides a common and easy framework for evaluation of your retrieval methods within the benchmark.

Refer to the repo via the link for a full list of supported datasets.

Here, we test the `all-MiniLM-L6-v2` sentence-transformer embedding, which is one of the fastest for the given accuracy range. We set the top_k value for the retriever to 30. We also use the nfcorpus dataset.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [5]:
%pip install llama-index-embeddings-huggingface

In [6]:
!pip install llama-index

In [8]:
!pip install beir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=854ab138f66c3b211ea9c4b377606664cc3a8ea0fa9a8c547b2a30a42173ef80
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
  Created wheel for p

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.evaluation.benchmarks import BeirEvaluator
from llama_index.core import VectorStoreIndex
import pandas as pd

# List of models to evaluate
models_to_test = [
    "BAAI/bge-small-en-v1.5",
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    "BAAI/bge-large-en-v1.5",
]

# Datasets for evaluation
datasets = ["nfcorpus"]

# Initialize results storage
results = []

# Evaluation function
def create_retriever(documents, model_name):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    index = VectorStoreIndex.from_documents(
        documents, embed_model=embed_model, show_progress=True
    )
    return index.as_retriever(similarity_top_k=30)

scores = {}

# Loop through models and evaluate
for model_name in models_to_test:
    print(f"Evaluating model: {model_name}")

    # Define a lambda function to pass the model_name dynamically
    retriever_creator = lambda documents: create_retriever(documents, model_name)

    # Run evaluation
    evaluator = BeirEvaluator()
    scores = evaluator.run(
        retriever_creator,
        datasets=datasets,
        metrics_k_values=[3, 10, 30]
    )

    # Store the model's scores
    scores["model"] = model_name
    results.append(scores)

# Create a DataFrame to compare models
df_results = pd.DataFrame(results)

# Display sorted results by the primary metric (e.g., NDCG@10)
df_sorted = df_results.sort_values(by="NDCG@10", ascending=False)
print(df_sorted)


Evaluating model: BAAI/bge-small-en-v1.5
Dataset: nfcorpus downloaded at: /tmp/llama_index/datasets/BeIR__nfcorpus
Evaluating on dataset: nfcorpus
-------------------------------------


  0%|          | 0/3633 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/3633 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1595 [00:00<?, ?it/s]

Retriever created for:  nfcorpus
Evaluating retriever on questions against qrels


100%|██████████| 323/323 [00:37<00:00,  8.67it/s]


Results for: nfcorpus
{'NDCG@3': 0.38766, 'MAP@3': 0.0847, 'Recall@3': 0.09833, 'precision@3': 0.37255}
{'NDCG@10': 0.3361, 'MAP@10': 0.11749, 'Recall@10': 0.15986, 'precision@10': 0.25325}
{'NDCG@30': 0.30161, 'MAP@30': 0.13719, 'Recall@30': 0.22052, 'precision@30': 0.15294}
-------------------------------------


TypeError: 'NoneType' object does not support item assignment

Higher is better for all the evaluation metrics.

This [towardsdatascience article](https://towardsdatascience.com/ranking-evaluation-metrics-for-recommender-systems-263d0a66ef54) covers NDCG, MAP and MRR in greater depth.